In [1]:
import ee
import geemap
from gee_0_utils import *

initialize()

config = ProjectConfig()
roi = config.roi
data_folder = config.data_folder
last_year = config.last_year

age, _, fire = desired_lulc()

## Fire

### Total number of fires per pixel

Each burned pixel contains the value of the land use type that burned (such as the desired_values vector specified in the land use section) - unburned pixels are zero.

There are instances of fires:
- after the first observation of anthropogenic land use
- before the first observation of anthropogenic land use (forest fire)
- after the last observation of anthropogenic land use (burn in regrowing forest)

*Note that fire has different transform than lulc, and the projections will be adjusted when exporting.*

### How many years ago was the last fire?

Make one image with 34 bands, each with the number of years from the instance of burn to 2020.
Each pixel is 0 if unburned, and "time since fire" in number of years if it's burned.

In [ ]:
# Load the FireCCI dataset
dataset = ee.ImageCollection('ESA/CCI/FireCCI/5_1')

# Define the list of years you want to process
years = list(range(2001, 2021))

# Function to calculate the burn count for a specific year
def annual_burn_count(year):
    year_collection = dataset.filterDate(f'{year}-01-01', f'{year}-12-31')
    burn_occurrences = year_collection.select('BurnDate').map(lambda img: img.gt(0).selfMask())
    return burn_occurrences.sum().rename(str(year))

# Map over the years to generate an image for each year's burn count
annual_burn_images = [annual_burn_count(year) for year in years]

fire = ee.Image.cat(annual_burn_images).byte().updateMask(age)
first_year_considered = 2001
years_regrowth_considered = range(2001, 2020)

In [ ]:


# Calculate burn information and add yearly bands to the corresponding images
for yr in years_regrowth_considered:
    yrs_ago = last_year - yr
    age_mask = age.eq(yrs_ago).selfMask()
    
    bands_after = fire.select([str(year) for year in range(yr, last_year+1)]).updateMask(age_mask)
    num_fires_after_regrowth = num_fires_after_regrowth.addBands(
    bands_after.reduce(ee.Reducer.sum()).byte().rename(f"{yrs_ago}"))

    # Add bands for before regrowth
    bands_before = fire.select([str(year) for year in range(first_year_considered, yr+1)]).updateMask(age_mask)
    num_fires_before_regrowth = num_fires_before_regrowth.addBands(
        bands_before.reduce(ee.Reducer.sum()).byte().rename(f"{yrs_ago}")
    )

# Remove the first band (assuming it's an unwanted initial band)
num_fires_before_regrowth = num_fires_before_regrowth.reduce(ee.Reducer.sum()).selfMask().unmask(0) \
    .rename("num_fires")

num_fires_after_regrowth = num_fires_after_regrowth.reduce(ee.Reducer.sum()).unmask(0) \
    .rename("num_fires_after_regrowth")
unburned_mask = num_fires_after_regrowth.eq(0).rename("unburned_mask")




In [ ]:

if ESA_fire:
    num_fires_before_regrowth = num_fires_before_regrowth.updateMask(unburned_mask)
    export_image(num_fires_before_regrowth, "ESA_fires", region = roi, scale = 30, folder = folder)
if method == "eu":
    num_fires_before_regrowth = num_fires_before_regrowth.updateMask(unburned_mask)
    # export_image(num_fires_before_regrowth, "eu_mapbiomas_fires", region = roi, scale = 30, folder = folder)
    export_image(unburned_mask, "mapbiomas_unburned_mask", region = roi, scale = 30, folder = folder)
else:
    export_image(land_use, f"land_use{name_LU}", region = roi, scale = 30, folder = folder)